In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense


In [2]:
train_data = pd.read_excel("Train.xlsx")
test_data = pd.read_excel("Test.xlsx")


In [3]:
data = pd.concat([train_data, test_data], axis=0)

In [15]:
input_parameters = ["timestamp"]
output_parameters = ["Temperature", "Precipitation", "Snowfall", "Totalcloudiness","Lowcloudcover", "Mediumcloudcover", "Highcloudcover","Pressure", "Windspeedm/s", "WindDirection", "WindGusts"]


In [16]:
data["timestamp"] = pd.to_datetime(data["timestamp"], format="%d.%m.%Y %H:%M:%S")
data.set_index("timestamp", inplace=True)
data.sort_index(inplace=True)


KeyError: 'timestamp'

In [17]:
data.fillna(method="ffill", inplace=True)


In [18]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)


In [19]:
# Number of previous timestamps to consider as input for predicting the next timestamp
window_size = 24
X = []
y = []
for i in range(len(data) - window_size):
    X.append(scaled_data[i:i + window_size])
    y.append(scaled_data[i + window_size])

X = np.array(X)
y = np.array(y)


In [20]:
train_size = len(train_data)
X_train = X[:train_size]
y_train = y[:train_size]
X_test = X[train_size:]
y_test = y[train_size:]


In [21]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True,input_shape=(window_size, len(data.columns))))
model.add(LSTM(units=50))
model.add(Dense(len(output_parameters)))


In [22]:
model.compile(optimizer='adam', loss='mean_squared_error')


In [23]:
model.fit(X_train, y_train, epochs=100, batch_size=32,validation_data=(X_test, y_test))


Epoch 1/100
1209/1209 [==============================] - 15s 11ms/step - loss: 0.0315 - val_loss: 0.0229
Epoch 2/100
1209/1209 [==============================] - 14s 11ms/step - loss: 0.0245 - val_loss: 0.0209
Epoch 3/100
1209/1209 [==============================] - 14s 11ms/step - loss: 0.0241 - val_loss: 0.0208
Epoch 4/100
1209/1209 [==============================] - 17s 14ms/step - loss: 0.0239 - val_loss: 0.0208
Epoch 5/100
1209/1209 [==============================] - 14s 11ms/step - loss: 0.0238 - val_loss: 0.0205
Epoch 6/100
1209/1209 [==============================] - 14s 11ms/step - loss: 0.0237 - val_loss: 0.0211
Epoch 7/100
1209/1209 [==============================] - 14s 11ms/step - loss: 0.0236 - val_loss: 0.0205
Epoch 8/100
1209/1209 [==============================] - 14s 11ms/step - loss: 0.0235 - val_loss: 0.0203
Epoch 9/100
1209/1209 [==============================] - 14s 11ms/step - loss: 0.0234 - val_loss: 0.0202
Epoch 10/100
1209/1209 [==============================]

In [24]:
predictions = model.predict(X_test)

# Transform predictions back to original scale
predictions = scaler.inverse_transform(predictions)

# Convert the predictions to a DataFrame with appropriate column names
output_df = pd.DataFrame(predictions, columns=output_parameters)

# Print the predicted output for testing dataset
print(output_df)


22/22 [==============================] - 1s 4ms/step
     Temperature  Precipitation  Snowfall  Totalcloudiness  Lowcloudcover  \
0      20.956734       0.030637  0.020178        36.064953      45.711761   
1      23.349775      -0.005239 -0.003315        32.841778      47.874405   
2      21.522356       0.025520 -0.006415        41.936920      60.739334   
3      18.926504       0.023920 -0.020239        43.028862      67.393402   
4      17.096247      -0.009363 -0.017671         6.445228      25.883717   
..           ...            ...       ...              ...            ...   
691    22.995884      -0.005001 -0.029042        38.776321      13.896963   
692    23.761736       0.024087 -0.018098        48.236118      13.543934   
693    23.574251       0.029769 -0.015880        37.909496       3.159430   
694    22.524378      -0.040267 -0.013402        34.212986       0.464174   
695    20.519035      -0.039720 -0.006623        19.870176       4.810752   

     Mediumcloudcover 